# Chess FEN → Score + Move Model
This notebook:

- Loads a `text` file where each line is `FEN|score_cp|other_cp|uci_move`.

- Splits it into four pandas columns.

- Encodes the FEN to numeric features (piece planes + side-to-move + castling + en passant info).

- Trains:

  1) A regressor to predict the first centipawn score.

  2) A classifier to predict the **exact** UCI move string.

- Evaluates and saves artifacts (`joblib` files + processed CSV).


> **Data path**: update `DATA_PATH` below if your file is different. Upload your data to `/mnt/data/`.



In [1]:
# === Setup ===
PROCESSED_PICKLE = "./fullDf.pkl"

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from typing import List, Tuple
# from collections import defaultdict

# Models
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score

import joblib

import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from keras import losses
from tqdm import tqdm


2025-09-17 17:25:50.318372: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 1) Load & split raw lines into columns

In [8]:
DATA_PATH = r"./useful_chess_data.txt"
assert os.path.exists(DATA_PATH), f"Data file not found at {DATA_PATH}. Upload your file there or change DATA_PATH."


maxLines = int(10**2 * 5.5)
amount = 38*10**6
idx = 0
lines = []
skipped = 0
with open(DATA_PATH, "r") as f:
    for line in tqdm(f,desc="Reading positions",total=38548203):
        line = line.strip()
        if line and idx%(amount//maxLines)==0:
            lines.append(line)
                # if (len(lines)%100000==0):
                #     print(f"Read {len(lines)}")
            if (len(lines)>=maxLines):
                print(f"Only using {len(lines)} lines")
                break
        idx+=1

df = pd.DataFrame([ln.split("|") for ln in lines], columns=["fen", "score_cp", "score2_cp", "uci"])
df["score_cp"] = pd.to_numeric(df["score_cp"], errors="coerce")
df["score2_cp"] = pd.to_numeric(df["score2_cp"], errors="coerce")

df.to_pickle("fullDf.pkl","zip")

print("Parsed chess data (preview)")
print(df.head(20))

AssertionError: Data file not found at ./useful_chess_data.txt. Upload your file there or change DATA_PATH.

In [9]:
df = pd.read_pickle("fullDf.pkl","zip")

In [ ]:
#Code to balance dataset by adding mirror of game but switching the colors
def reverse_fen(fen):
    board, turn, *rest = fen.split(" ")
    # 1. Swap piece colors
    swapped = "".join(
        c.lower() if c.isupper() else c.upper() if c.islower() else c
        for c in board
    )
    # 2. Reverse ranks
    ranks = swapped.split("/")
    reversed_board = "/".join(ranks[::-1])
    # 3. Flip turn
    new_turn = "w" if turn == "b" else "b"
    # 4. Rebuild FEN
    new_fen = " ".join([reversed_board, new_turn] + rest)
    # 5. Flip eval
    return new_fen

testFen = "4k3/3pppp1/8/8/8/8/8/3QK3 w - - 0 1"
print(f'Reversing fen "{testFen}" to "{reverse_fen(testFen)}"')

#Augment dataframe
def augmentDf(df):
    print("Original size:", len(df))
    augmented_rows = []
    for _, row in tqdm(df.iterrows(),total=len(df),desc="Equalizing dataframe",colour="green",ncols=100):
        newFen = reverse_fen(row["fen"])
        augmented_rows.append({
            "fen" : newFen,
            "score_cp" : row["score_cp"],
            "score2_cp" : row["score2_cp"],
            "uci" : "a1a1",
        })
    df_aug = pd.DataFrame(augmented_rows)
    print("Augmented size:", len(df_aug)+len(df))
    return pd.concat([df, df_aug], ignore_index=True)

df = augmentDf(df)

3qk3/8/8/8/8/8/3PPPP1/4K3 b - - 0 1
Original size: 550000


Equalizing dataframe: 100%|███████████████████████████████| 550000/550000 [01:09<00:00, 7867.39it/s]


Augmented size: 1100000


In [ ]:
#Search df for black positions with high centipawn evaluations
def searchDf(df):
    blackRows = []
    for _, row in tqdm(df.iterrows(),total=len(df),desc="Searching dataframe",colour="green",ncols=100):
        if row["fen"].split()[1] == "b" and row["uci"] != "a1a1" and abs(row["score_cp"]) >= 400:

            blackRows.append({
                "fen" : row["fen"],
                "score_cp" : row["score_cp"],
                "score2_cp" : row["score2_cp"],
                "uci" : row["uci"],
            })
    print(len(blackRows))
    print(*blackRows,sep="\n")

searchDf(df)

## 2) FEN encoder
We encode each board into a feature vector: 64 squares × 12 piece planes (P,N,B,R,Q,K for white/black), plus side-to-move, castling rights (KQkq), and en passant file.

We can also encode the FEN as an "Image" with 12 channels and add a couple of extra bits of information afterwards

In [39]:
PIECE_TO_PLANE = {
    'P':0,'N':1,'B':2,'R':3,'Q':4,'K':5,
    'p':6,'n':7,'b':8,'r':9,'q':10,'k':11
}

def fen_to_feature_vector(fen: str) -> np.ndarray:
    # FEN: <board> <side> <castling> <enpassant> <halfmove> <fullmove>
    parts = fen.strip().split()
    assert len(parts) >= 4, f"Bad FEN: {fen}"
    board, side, castling, ep = parts[0], parts[1], parts[2], parts[3]

    # Board: 8 ranks separated by '/'; each rank has pieces or digits for empty squares
    planes = np.zeros((12, 8, 8), dtype=np.float32)
    ranks = board.split('/')
    assert len(ranks) == 8, f"Bad board: {board}"
    for r, rank in enumerate(ranks):
        file_idx = 0
        for ch in rank:
            if ch.isdigit():
                file_idx += int(ch)
            else:
                if ch in PIECE_TO_PLANE:
                    plane = PIECE_TO_PLANE[ch]
                    planes[plane, r, file_idx] = 1.0
                    file_idx += 1
                else:
                    raise ValueError(f"Unknown piece char: {ch}")

    features = []
    features.extend(planes.reshape(-1).tolist())  # 12*8*8 = 768

    # Side to move: 1 for white, 0 for black
    features.append(1.0 if side == 'w' else 0.0)

    # Castling rights: K, Q, k, q flags
    features.append(1.0 if 'K' in castling else 0.0)
    features.append(1.0 if 'Q' in castling else 0.0)
    features.append(1.0 if 'k' in castling else 0.0)
    features.append(1.0 if 'q' in castling else 0.0)

    # En passant file (a-h → 0-7), or - if none
    ep_file = -1
    if ep != '-':
        file_letter = ep[0]
        if file_letter in "abcdefgh":
            ep_file = "abcdefgh".index(file_letter)
    # One-hot encode ep file into length 9 (none + 8 files)
    for i in range(8):
        features.append(1.0 if ep_file == i else 0.0)
    features.append(1.0 if ep_file == -1 else 0.0)  # none flag

    return np.array(features, dtype=np.float32)

def fen_to_image_and_extra(fen: str):
    parts = fen.strip().split()
    board, side, castling, ep = parts[0], parts[1], parts[2], parts[3]

    img = np.zeros((8,8,12), dtype=np.float32)
    ranks = board.split('/')
    for r, rank in enumerate(ranks):
        file_idx = 0
        for ch in rank:
            if ch.isdigit():
                file_idx += int(ch)
            else:
                img[r, file_idx, PIECE_TO_PLANE[ch]] = 1.0
                file_idx += 1

    # Extra bits
    extras = []
    extras.append(1.0 if side == 'w' else 0.0)
    extras.append(1.0 if 'K' in castling else 0.0)
    extras.append(1.0 if 'Q' in castling else 0.0)
    extras.append(1.0 if 'k' in castling else 0.0)
    extras.append(1.0 if 'q' in castling else 0.0)

    ep_bits = [0.0]*8
    if ep != '-' and ep[0] in "abcdefgh":
        ep_bits["abcdefgh".index(ep[0])] = 1.0
    extras.extend(ep_bits)

    return img, np.array(extras, dtype=np.float32)

def fen_to_flat(fen: str, debug=False):
    parts = fen.strip().split()
    board, side, castling, ep = parts[0], parts[1], parts[2], parts[3]
    img = np.zeros((8,8,12), dtype=np.float16)
    ranks = board.split('/')
    for r, rank in enumerate(ranks):
        file_idx = 0
        for ch in rank:
            if ch.isdigit():
                print(f"Read {ch} file_idx is now {file_idx}")
                file_idx += int(ch)
            else:
                if (PIECE_TO_PLANE[ch] == 10):
                    print(f"Piece {ch} is on position {r,file_idx}")
                img[r, file_idx, PIECE_TO_PLANE[ch]] = 1.0
                file_idx += 1

    if (debug):
        print("8 x 8 x 12, without extras")
        print(*img)

    features = []
    features.extend(img.reshape(-1).tolist())

    features.append(1.0 if side == 'w' else 0.0)

    # return img.flatten()
    return np.array(features, dtype=np.float16)

# Quick sanity check
x = fen_to_feature_vector(df.iloc[0]['fen'])
print('Feature length of vector:', x.shape[0])

x = fen_to_image_and_extra(df.iloc[0]["fen"])
print(f"Feature length of image {x[0].shape} and {x[1].shape}")

x = fen_to_flat(df.iloc[0]["fen"])
print(f"Feature length of flat {x.shape}")


Feature length of vector: 782
Feature length of image (8, 8, 12) and (13,)
Read 1 file_idx is now 0
Read 1 file_idx is now 2
Read 1 file_idx is now 4
Read 2 file_idx is now 6
Read 1 file_idx is now 0
Read 1 file_idx is now 2
Read 1 file_idx is now 4
Read 2 file_idx is now 6
Read 7 file_idx is now 0
Read 5 file_idx is now 1
Read 1 file_idx is now 7
Read 1 file_idx is now 1
Read 3 file_idx is now 3
Read 1 file_idx is now 7
Read 7 file_idx is now 0
Read 1 file_idx is now 0
Read 1 file_idx is now 2
Read 4 file_idx is now 4
Read 8 file_idx is now 0
Feature length of flat (769,)


In [ ]:
#Little test to understand how flatening works in np
test_np = np.zeros((8,8,12), dtype=np.float32)
for i in range(8):
    for j in range(8):
        for k in range(12):
            test_np[i,j,k] = i*10000+j*100+k

test_features = []
test_features.extend(test_np.reshape(-1).tolist())
for idx,val in enumerate(test_features):
    print(f"Idx:{idx} val:{val}")

## 3) Build feature matrix X and target y
- `y_score` = first centipawn score (`score_cp`)

In [ ]:
# Build X features
X = np.stack(df['fen'].apply(fen_to_feature_vector).values)


print('X shape:', X.shape)

# Train/validation split
X_train, X_test = train_test_split(
    X, test_size=0.2, random_state=42
)


X shape: (200000, 782)
Sample y_score: [ -34. -109.  -33.  -33.  -89.]


In [11]:
y_score = df['score_cp'].values.astype(np.float16)

### Prepare data for NN

In [12]:
board_imgs = []
for fen,static_score in tqdm(zip(df["fen"],df["score2_cp"]),total=len(df),desc="Formating NN training and validation data",colour="green"):
    img = fen_to_flat(fen)
    img = np.append(img, np.float16(static_score))
    board_imgs.append(img)

board_imgs = np.array(board_imgs)

imgs_train, imgs_test, y_score_train, y_score_test = train_test_split(
    board_imgs, y_score, test_size=0.2, random_state=42
)
y_score_train = y_score_train.reshape(-1, 1).astype("float16")
y_score_test  = y_score_test.reshape(-1, 1).astype("float16")

Formating NN training and validation data: 100%|██████████| 550000/550000 [00:55<00:00, 9891.88it/s] 


## 4) Models
- **Score regressor**: Ridge regression
- **Score preditor**: NN with tensorflow

In [ ]:
def build_chess_cnn():
    # board_input = Input(shape=(8,8,12), name="board")
    # x = layers.Conv2D(16, (3,3), activation="relu", padding="same")(board_input)
    # x = layers.Dropout(0.25)(x)
    # x = layers.Conv2D(16, (2,2), activation="relu", padding="same")(x)
    # x = layers.Dropout(0.25)(x)
    # x = layers.Flatten()(x)


    # extra_input = Input(shape=(n_extra,), name="extra")
    # merged = layers.concatenate([x, extra_input])
    # merged = layers.Dense(16, activation="relu")(board_input)
    # merged = layers.Dropout(0.5)(merged)

    board_input = Input(shape=(770,), name="board")
    # hidden = layers.Dense(32, activation="relu",kernel_regularizer=l2(0.001))(board_input)
    # hidden = layers.Dropout(0.2)(hidden)
    hidden = layers.Dense(32, activation="relu",kernel_regularizer=l2(0.001))(board_input)
    for i in range(7):
        hidden = layers.Dense(16, activation="relu",kernel_regularizer=l2(0.001))(hidden)
    # hidden = layers.Dropout(0.20)(hidden)
    value_out = layers.Dense(1, name="value")(hidden)


    # model = models.Model(inputs=[board_input, extra_input], outputs=[value_out])
    model = models.Model(inputs=[board_input], outputs=[value_out])
    model.compile(
    optimizer="adam",
    loss={"value": "mse"},
    # loss = losses.Huber(delta=5.0), 
    metrics={"value": "mae"}
    )
    return model


#A NN just ot see if it works
model = build_chess_cnn()
model.summary()

In [ ]:
print(type(imgs_train[0][0]))
print(type(y_score_train[0]))

print(imgs_train.shape, imgs_train.dtype)
print(imgs_test.shape, imgs_test.dtype)
print(y_score_train.shape, y_score_train.dtype)
print(y_score_test.shape, y_score_test.dtype)

print("Any NaNs in imgs_train?", np.isnan(imgs_train).any())
print("Any NaNs in y_score_train?", np.isnan(y_score_train).any())
print("Any strings in y_score_train?", any(isinstance(x, str) for x in y_score_train))
print("Any strings in board_imgs?", any(isinstance(x, str) for x in board_imgs))
print("Any strings in y_score?", any(isinstance(x, str) for x in y_score))

print(board_imgs[0])

In [41]:
#Arreter en cas d'overfitting ou de stagnation
early_stop = EarlyStopping(monitor = "val_loss", patience = 5, restore_best_weights=True)

#Permettre qu'il continue meme si plus lentement
reduce_lr = ReduceLROnPlateau(monitor = "val_loss", mode = "min", factor = 0.5, patience = 3)

#Train CNN model
history = model.fit(
    imgs_train,
    y_score_train,
    # validation_data=({"board": imgs_test, "extra": extras_test}, {"value": y_score_test),
    validation_data=(imgs_test,y_score_test),
    epochs=100,
    batch_size=2048,
    verbose=1,
    callbacks = [early_stop, reduce_lr]
)

Epoch 1/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - loss: 866.8355 - mae: 22.6789 - val_loss: 850.5442 - val_mae: 22.4630 - learning_rate: 2.4414e-07
Epoch 2/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 852.4680 - mae: 22.4791 - val_loss: 841.2231 - val_mae: 22.3331 - learning_rate: 2.4414e-07
Epoch 3/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 842.6504 - mae: 22.3363 - val_loss: 835.0042 - val_mae: 22.2456 - learning_rate: 2.4414e-07
Epoch 4/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - loss: 838.1335 - mae: 22.2499 - val_loss: 830.0657 - val_mae: 22.1740 - learning_rate: 2.4414e-07
Epoch 5/100


KeyboardInterrupt: 

In [ ]:
print(history.history.keys())
# plt.plot(history.history['loss'], label="Train loss")
# plt.plot(history.history['val_loss'], label="Validation loss")
plt.plot(history.history["mae"], label="MAE")
plt.plot(history.history["val_mae"], label="Validation MAE")
plt.legend()
plt.show()

In [13]:
#Test CNN model
test_results = model.evaluate(
    # {"board": imgs_test, "extra": extras_test},
    imgs_test,
    y_score_test,
    batch_size=1024,
    verbose=1
)

#Best result
# [696.7526245117188, 19.979074478149414]
#Best on doubled
# [754.0731201171875, 21.048927307128906]
# [833.1061401367188, 22.360122680664062]
print("Test results:", test_results)

215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 894.3179 - mae: 23.0600
Test results: [892.817626953125, 23.034631729125977]


In [40]:
def predict_eval(fen,static_eval):
    x_input = fen_to_flat(fen,debug=False)
    x_input = np.append(x_input, np.float32(static_eval))
    print(*x_input,sep=",")
    x_input = x_input.reshape(1, -1)

    print(x_input.shape)
    prediction = model.predict(x_input)
    print(f"Predicted evaluation (centipawns) for '{fen}' : {prediction[0][0]:.3f}")

predict_eval("r2qk4/8/8/8/8/8/PPPPPPPP/RNBQKBNR w KQq - 0 1",2.390)
predict_eval("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1",0.0)

Read 2 file_idx is now 1
Piece q is on position (0, 3)
Read 4 file_idx is now 5
Read 8 file_idx is now 0
Read 8 file_idx is now 0
Read 8 file_idx is now 0
Read 8 file_idx is now 0
Read 8 file_idx is now 0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [ ]:
import random
nb_tests = 100
totalError = 0
for i in range(nb_tests):
    wantedIdx = random.randint(0,10**5)
    x_input = fen_to_flat(df["fen"][wantedIdx])
    x_input = np.append(x_input, np.float32(df["score2_cp"][wantedIdx]))
    x_input = x_input.reshape(1, -1)

    print(x_input.shape)
    prediction = model.predict(x_input)
    error = int(df['score_cp'][wantedIdx])-(prediction[0][0])
    totalError+=abs(error)
    # print(f"Predicted evaluation (centipawns) for '{df['fen'][wantedIdx]}' : {prediction[0][0]:.3f} accurate is {df['score_cp'][wantedIdx]}, error is {error:.3f}")
print(f"Mean absolute error = {totalError/nb_tests}")

In [5]:
# model.save("test.keras")
model.export("test")


INFO:tensorflow:Assets written to: test/assets


INFO:tensorflow:Assets written to: test/assets


Saved artifact at 'test'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 770), dtype=tf.float32, name='board')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  5150825056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5150826112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5150825760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5150826640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5150826288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5150827168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5150826992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5150827520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5150827344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5150827872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5150827696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5150828224: TensorSpec(sha

In [3]:
model = tf.keras.models.load_model("best_on_doubled.keras")  
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ board (InputLayer)              │ (None, 770)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 32)             │        24,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ value (Dense)                   │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 80,549 (314.65 KB)

 Trainable params: 26,849 (104.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 53,700 (209.77 KB)

In [ ]:
# Regressor for score
regressor = Ridge(alpha=1, random_state=42)
regressor.fit(X_train, y_score_train)
print(f"Fitted model for score")

# Evaluate
y_score_pred = regressor.predict(X_test)
mae = mean_absolute_error(y_score_test, y_score_pred)
r2 = r2_score(y_score_test, y_score_pred)

#Best 27.44
print(f"Score MAE (cp): {mae:.2f}")
#Best 0.909
print(f"Score R^2: {r2:.3f}")

# Save artifacts
joblib.dump(regressor, REGRESSOR_PATH)
print("Saved regressor to disk.")

## 5) Inference helper
`predict_score_and_move(fen)` → `(score_cp_pred, uci_move_pred)`

In [ ]:
def predict_score_and_move(fen: str) -> float:
    x = fen_to_feature_vector(fen).reshape(1, -1)
    score_pred = float(regressor.predict(x)[0])
    return score_pred

# Demo with first row
demo_score = predict_score_and_move(df.iloc[0]['fen'])
print('Demo prediction:', demo_score)


## 6) Export: metrics + sample predictions

In [6]:
# Collect some sample predictions for inspection
n_show = min(10, len(X_test))
sample_rows = []
for i in range(n_show):
    fen = df.iloc[i]['fen']
    true_score = df.iloc[i]['score_cp']
    true_move = df.iloc[i]['uci']
    ps = predict_score_and_move(fen)
    sample_rows.append({
        "fen": fen,
        "true_score_cp": true_score,
        "pred_score_cp": round(ps, 2),
    })
samples_df = pd.DataFrame(sample_rows)
samples_df


NameError: name 'X_test' is not defined